In [ ]:
import numpy as np 
import pandas as pd 

from statistics import mean

import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.svm import SVC 

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
train_text = pd.read_csv("train2.csv")

In [ ]:
y_cols = ["support","storage","aruba", "devices-sub", "Services"]
train_text[y_cols].apply(pd.Series.value_counts, args = (True, True, False, None, False))

In [ ]:
X = train_text["query"]
y = train_text[["support","storage","aruba", "devices-sub", "Services"]]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = y.apply(le.fit_transform)
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2,random_state=0)

In [ ]:
stop_words = _stop_words.ENGLISH_STOP_WORDS
def clean(doc):
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stop_words])
    return doc.lower()

In [ ]:
vect = TfidfVectorizer(max_features= 3000, preprocessor=clean)
X_train_dtm = vect.fit_transform(X_train)
X_val_dtm = vect.transform(X_val)

In [ ]:
pd.DataFrame(X_train_dtm.A[:5], columns = vect.get_feature_names_out())

In [ ]:

model = OneVsRestClassifier(SVC(probability=True))
lr = model.fit(X_train_dtm, y_train)

In [ ]:
def calculate_roc_auc(y_test, y_pred):
    aucs = []
    #Calculate the ROC-AUC for each of the target column
    for col in range(y_test.shape[1]):
        aucs.append(roc_auc_score(y_test[:,col],y_pred[:,col]))
    return aucs

In [ ]:
test_text = pd.read_csv("test1.csv")

In [ ]:
X_test_dtm = vect.transform(test_text["query"])
y_preds = lr.predict_proba(X_test_dtm)
# y_preds = np.transpose(np.array(lr.predict_proba(X_test_dtm)))
# #Assign the predictions by the model in the final test dataset

test_text[y_cols] = y_preds

test_text

In [ ]:
column_names = ["support","storage","aruba", "devices-sub", "Services"]
test_query = "VG2406042959"
X_test_dtm = vect.transform([test_query])
#Use the Logistic Regression model to output probabilities and take the probability for class 1
y_preds = lr.predict_proba(X_test_dtm)
# y_preds = np.transpose(np.array(lr.predict_proba(X_test_dtm))[:,:,1])
res = pd.DataFrame(y_preds, index=[test_query], columns=column_names)
res